<a href="https://colab.research.google.com/github/plushvoxel/Project-Lernende-Agenten-colab/blob/master/CNNTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
import numpy as np
import tensorflow as tf
from tarfile import open as taropen
from urllib import request
from struct import unpack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [0]:
def create_data(trainP = 60, valP= 20, testP = 20):
    pre: trainP + valP + testP <= 100
    DataBaseURL= "http://computer-in.love/data/{}.tar"
    Classes = ["fm", "pager", "smartwares", "no_signal"]
    data = []
    normalisationFactor = 100
    test = 0

    for modulation in Classes:
        filename = "{}.tar".format(modulation)
        request.urlretrieve(DataBaseURL.format(modulation), filename)
        tar = taropen(filename)
        class_label = [0]*len(Classes)
        class_label[Classes.index(modulation)]=1
        for member in tar.getmembers():
            test = test + 1
            with tar.extractfile(member) as f:
                sample = []
                buffer = f.read()
                num_floats = len(buffer)//4
                floats = unpack("f"*num_floats, buffer)
                i = floats[0::2]
                q = floats[1::2]
                for j in range(min(len(i), len(q))):
                    #here happens the scaling or whatever you want
                    sample.append([i[j]/normalisationFactor, q[j]/normalisationFactor])
                #here happens some cross_feature if you want. just append it to the sample
                
                data.append([np.array(sample), np.array(class_label)])

    random.shuffle(data)
    #split all data into train, val and test
    size =len(data)
    trainSize = (size * trainP)//100
    print(trainSize)
    valSize = (size * valP)//100
    print(valSize)
    testSize = (size * testP)//100
    print(testSize)
    train = data[:trainSize]
    val = data[trainSize: trainSize + valSize]
    test = data[trainSize + valSize: trainSize + valSize + testSize]
    x_train = []
    y_train = []
    x_val = []
    y_val = []
    x_test = []
    y_test = []
    for sample, target in train:
      x_train.append(sample)
      y_train.append(target)
    for sample, target in val:
      x_val.append(sample)
      y_val.append(target)
    for sample, target in test:
      x_test.append(sample)
      y_test.append(target)
    
    
    return np.array(x_train), np.array(x_val), np.array(x_test), np.array(y_train), np.array(y_val), np.array(y_test) 

In [3]:
train_X, val_X, test_X, train_y, val_y, test_y = create_data()


189600
63200
63200


In [4]:
print(np.max(train_X[0]))

0.3818700408935547


In [0]:
train_X = train_X.reshape(-1, 128, 2, 1)
test_X = test_X.reshape(-1,128,2,1)

In [0]:
training_iters = 1 
learning_rate = 0.001 
batch_size = 8
#data input (img shape: 128*2)
n_input_1 = 128
n_input_2 = 2

# data total classes (0-9 digits)
n_classes = 4

#both placeholders are of type float
x = tf.placeholder("float", [None, n_input_1,n_input_2,1])
y = tf.placeholder("float", [None, n_classes])

In [0]:
weights = {
    'wc1': tf.get_variable('W0', shape=(11,2,1,8), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc2': tf.get_variable('W1', shape=(11,2,8,16), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc3': tf.get_variable('W2', shape=(11,2,16,32), initializer=tf.contrib.layers.xavier_initializer()), 
    'wd1': tf.get_variable('W3', shape=(128*32,32), initializer=tf.contrib.layers.xavier_initializer()), 
    'out': tf.get_variable('W6', shape=(32,n_classes), initializer=tf.contrib.layers.xavier_initializer()), 
}
biases = {
    'bc1': tf.get_variable('B0', shape=(8), initializer=tf.contrib.layers.xavier_initializer()),
    'bc2': tf.get_variable('B1', shape=(16), initializer=tf.contrib.layers.xavier_initializer()),
    'bc3': tf.get_variable('B2', shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
    'bd1': tf.get_variable('B3', shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
    'out': tf.get_variable('B4', shape=(n_classes), initializer=tf.contrib.layers.xavier_initializer()),
}

In [0]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x) 

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, 1, 1], strides=[1, k, k, 1],padding='SAME')

In [0]:
def conv_net(x, weights, biases):  

    # here we call the conv2d function we had defined above and pass the input image x, weights wc1 and bias bc1.
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*1 matrix window and outputs a 64*2 matrix.
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    # here we call the conv2d function we had defined above and pass the input image x, weights wc2 and bias bc2.
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*1 matrix window and outputs a 32*2 matrix.
    conv2 = maxpool2d(conv2, k=2)

    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*1 matrix window and outputs a 16*2.
    conv3 = maxpool2d(conv3, k=2)


    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Output, class prediction
    # finally we multiply the fully connected layer with the weights and add a bias term. 
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [0]:
pred = conv_net(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

#Here you check whether the index of the maximum value of the predicted image is equal to the actual labelled image. and both will be a column vector.
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))

#calculate accuracy across all the given images and average them out. 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [0]:
init = tf.global_variables_initializer()

In [12]:
with tf.Session() as sess:
    sess.run(init) 
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    for i in range(training_iters):
        #for batch in range(len(train_X)//batch_size):
        for batch in range(2):
            batch_x = train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
            batch_y = train_y[batch*batch_size:min((batch+1)*batch_size,len(train_y))]    
            # Run optimization op (backprop).
                # Calculate batch loss and accuracy
            opt = sess.run(optimizer, feed_dict={x: batch_x,
                                                              y: batch_y})
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y})
        print("Iter " + str(i) + ", Loss= " + \
                      "{:.6f}".format(loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(acc))
        print("Optimization Finished!")

        # Calculate accuracy for all 10000 mnist test images
        print(test_X.shape)
        print(test_y.shape)
        test_acc,valid_loss = sess.run([accuracy,cost], feed_dict={x: test_X,y : test_y})
        train_loss.append(loss)
        test_loss.append(valid_loss)
        train_accuracy.append(acc)
        test_accuracy.append(test_acc)
        print("Testing Accuracy:","{:.5f}".format(test_acc))
    summary_writer.close()
    plt.plot(range(len(train_loss)), train_loss, 'b', label='Training loss')
    plt.plot(range(len(train_loss)), val_loss, 'r', label='Test loss')
    plt.title('Training and Test loss')
    plt.xlabel('Epochs ',fontsize=16)
    plt.ylabel('Loss',fontsize=16)
    plt.legend()
    plt.figure()
    plt.show()
    plt.plot(range(len(train_loss)), train_accuracy, 'b', label='Training Accuracy')
    plt.plot(range(len(train_loss)), val_accuracy, 'r', label='Test Accuracy')
    plt.title('Training and Test Accuracy')
    plt.xlabel('Epochs ',fontsize=16)
    plt.ylabel('Loss',fontsize=16)
    plt.legend()
    plt.figure()
    plt.show()

Iter 0, Loss= 1.340818, Training Accuracy= 0.50000
Optimization Finished!
(63200, 128, 2, 1)
(63200, 4)


InvalidArgumentError: ignored